In [1]:
import tensorflow as tf
import numpy as np
import h5py
import scipy
from skimage.util.montage import montage2d
from skimage.color import label2rgb
import os
from sklearn.metrics import precision_score, recall_score
%matplotlib inline

In [2]:
ct_data, pet_data, label_data = [], [], []
num_data = 7
with h5py.File(os.path.join('..','input','lab_petct_vox_5.00mm.h5'),'r') as p_data:
    for ct_img, pet_img, label in zip(p_data['ct_data'].values(),
                                      p_data['pet_data'].values(),
                                      p_data['label_data'].values()):
        ct_data += [np.array(ct_img)]
        pet_data += [np.array(pet_img)]
        label_data += [np.array(label)]
        num_data = num_data - 1
        if num_data == 0:
            break
label_data[2][20:52] = label_data[2][90:122]
pet_data[2][20:52] = pet_data[2][90:122]
del label_data[5]
del label_data[4]

In [11]:
# Parameters
side = [75,78,96]
training_iters = 500
batch_size = 1
display_step = 10

# Network Parameters
n_input = (175,1,100)
n_classes = 2 
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [1, side[0], side[1], side[2], 1])
#x = tf.placeholder(tf.float32, [1, 175, 100, 100, 1])
#y = tf.placeholder(tf.float32, [175,100,100])
y = tf.placeholder(tf.float32, [side[0],side[1],side[2]])
y_flat = tf.reshape(y, [-1])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


# Create some wrappers for simplicity
def conv3d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv3d(x, W, strides=[1, strides, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool3d(x, k=2):
    # MaxPool3D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, k, 1], strides=[1, k, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):

    # Convolution Layer
    conv1 = conv3d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    #conv1 = maxpool3d(conv1, k=2)

    # Convolution Layer
    # conv2 = conv3d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    #conv2 = maxpool3d(conv2, k=2)

    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input
    #fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    #fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    #fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    #fc1 = tf.nn.dropout(fc1, dropout)
    #conv3 = conv3d(conv2, weights['wd1'], biases['bd1'])
    # Output, class prediction
    #out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return tf.reshape(conv1, [-1])

# Store layers weight & bias
weights = {
    # 16x16x16 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([2, 2, 2, 1, 1])),
    # 16x16 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([16, 16, 16, 10, 1])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([16, 16, 16, 64, n_classes])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([1])),
    'bc2': tf.Variable(tf.random_normal([1])),
    'bd1': tf.Variable(tf.random_normal([1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(y_flat, pred, pos_weight=1000))
global_step = tf.Variable(0, trainable=False)
learning_rate = 0.1
momentum = 5.0
learning_rate = tf.train.exponential_decay(.1, global_step,
                                           100000, 0.96, staircase=True)
momentum = tf.train.exponential_decay(1.0, global_step,
                                           100000, 0.96, staircase=True)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum, use_nesterov=True).minimize(cost)

# Evaluate model
#correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_flat, 1))
y_pred = tf.sigmoid(pred)
y_actual = y_flat
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [5]:
#la = []
#for i in range(4):
##    l = np.expand_dims(label_data[i], axis=4)
#    l = np.concatenate([np.zeros((175,100,100,1)), l], axis = 3)
#    for i in range(l.shape[0]):
##        for j in range(l.shape[1]):
#            for k in range(l.shape[2]):
#                if l[i,j,k,0] == 0:
#                    l[i,j,k] = [1,0]
#                else:
#                    l[i,j,k] = [0,1]
#   la += [l]
    
        

        

for i in range(5):
    label_data[i] = label_data[i][:75,:78,:96]
    pet_data[i] = pet_data[i][:75,:78,:96]
    np.random.seed(1234)
    np.random.shuffle(label_data[i])
    np.random.seed(1234)
    np.random.shuffle(label_data[i])    
    for x,y in zip(label_data[i], pet_data[i]):
        np.random.seed(1234)
        np.random.shuffle(x)
        np.random.seed(1234)
        np.random.shuffle(y)
        for a,b in zip(x,y):
            np.random.seed(1234)
            np.random.shuffle(a)
            np.random.seed(1234)
            np.random.shuffle(b)
            


In [6]:
print(np.max(np.where(label_data[0]>0),axis=1))
print(np.max(np.where(label_data[1]>0),axis=1))
print(np.max(np.where(label_data[2]>0),axis=1))
print(np.max(np.where(label_data[3]>0),axis=1))
print(np.max(np.where(label_data[4]>0),axis=1))




[62 64 80]
[72 67 93]
[74 77 94]
[74 77 93]
[74 77 90]


In [12]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < 1000:
        batch_x, batch_y = [np.expand_dims(pet_data[step%5], axis=4)], label_data[step%5]  # mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % 1 == 0:
            # Calculate batch loss and accuracy
            loss, y_p, y_a = sess.run([cost, y_pred, y_actual], feed_dict={x: batch_x,
                                                              y: batch_y})
            try:    
                prec = precision_score(y_a>0, y_p>.5)
                rec = recall_score(y_a>0, y_p>.5)
                #print(y_p[np.where(y_p>.5)])
                #print(np.shape(np.where(y_p>5))[0])
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Precision= " + \
                      "{:.5f}".format(prec) + ", Training Recall= " + \
                      "{:.5f}".format(rec))
            except Exception:
                import traceback
                traceback.print_exc()
                #print(str(step*batch_size) + " " + str(loss))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
    #                                  y: mnist.test.labels[:256],
    #                                  keep_prob: 1.}))

C:\Users\Himanshu\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Iter 1, Minibatch Loss= 0.696506, Training Precision= 0.00000, Training Recall= 0.00000
Iter 2, Minibatch Loss= 0.705935, Training Precision= 0.00000, Training Recall= 0.00000
Iter 3, Minibatch Loss= 0.695530, Training Precision= 0.00000, Training Recall= 0.00000
Iter 4, Minibatch Loss= 0.703668, Training Precision= 0.00000, Training Recall= 0.00000
Iter 5, Minibatch Loss= 0.694467, Training Precision= 0.00000, Training Recall= 0.00000
Iter 6, Minibatch Loss= 0.696506, Training Precision= 0.00000, Training Recall= 0.00000
Iter 7, Minibatch Loss= 0.705935, Training Precision= 0.00000, Training Recall= 0.00000
Iter 8, Minibatch Loss= 0.695530, Training Precision= 0.00000, Training Recall= 0.00000
Iter 9, Minibatch Loss= 0.703668, Training Precision= 0.00000, Training Recall= 0.00000
Iter 10, Minibatch Loss= 0.694467, Training Precision= 0.00000, Training Recall= 0.00000
Iter 11, Minibatch Loss= 0.696506, Training Precision= 0.00000, Training Recall= 0.00000
Iter 12, Minibatch Loss= 0.705

In [40]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < 100:
        batch_x, batch_y = [np.expand_dims(pet_data[step%5], axis=4)], label_data[step%5]  # mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if True:
            # Calculate batch loss and accuracy
            loss, y_p, y_a = sess.run([cost, y_pred, y_actual], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            try:    
                prec = precision_score(y_a>0, y_p>.5)
                rec = recall_score(y_a>0, y_p>.5)
                #print(y_p[np.where(y_p>.5)])
                #print(np.shape(np.where(y_p>5))[0])
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Precision= " + \
                      "{:.5f}".format(prec) + ", Training Recall= " + \
                      "{:.5f}".format(rec))
            except Exception:
                import traceback
                traceback.print_exc()
                #print(str(step*batch_size) + " " + str(loss))
        step += 1
    print("Optimization Finished!")


TypeError: unhashable type: 'numpy.ndarray'

In [20]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < 100:
        batch_x, batch_y = [np.expand_dims(pet_data[step%5], axis=4)], label_data[step%5]  # mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if True:
            # Calculate batch loss and accuracy
            loss, y_p, y_a = sess.run([cost, y_pred, y_actual], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            try:    
                prec = precision_score(y_a>0, y_p>.5)
                rec = recall_score(y_a>0, y_p>.5)
                #print(y_p[np.where(y_p>.5)])
                #print(np.shape(np.where(y_p>5))[0])
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Precision= " + \
                      "{:.5f}".format(prec) + ", Training Recall= " + \
                      "{:.5f}".format(rec))
            except Exception:
                import traceback
                traceback.print_exc()
                #print(str(step*batch_size) + " " + str(loss))
        step += 1
    print("Optimization Finished!")


Iter 1, Minibatch Loss= 114.374886, Training Precision= 0.00479, Training Recall= 0.96302
Iter 2, Minibatch Loss= 206.438812, Training Precision= 0.01761, Training Recall= 0.90964
Iter 3, Minibatch Loss= 94.598145, Training Precision= 0.00495, Training Recall= 0.94432
Iter 4, Minibatch Loss= 1.790671, Training Precision= 0.00716, Training Recall= 0.01852
Iter 5, Minibatch Loss= 73.261185, Training Precision= 0.00149, Training Recall= 0.58837
Iter 6, Minibatch Loss= 33.987537, Training Precision= 0.00539, Training Recall= 0.41077


KeyboardInterrupt: 

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < 100:
        batch_x, batch_y = [np.expand_dims(pet_data[step%5], axis=4)], label_data[step%5]  # mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if True:
            # Calculate batch loss and accuracy
            loss, y_p, y_a = sess.run([cost, y_pred, y_actual], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            try:    
                prec = precision_score(y_a>0, y_p>.5)
                rec = recall_score(y_a>0, y_p>.5)
                #print(y_p[np.where(y_p>.5)])
                #print(np.shape(np.where(y_p>5))[0])
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Precision= " + \
                      "{:.5f}".format(prec) + ", Training Recall= " + \
                      "{:.5f}".format(rec))
            except Exception:
                import traceback
                traceback.print_exc()
                #print(str(step*batch_size) + " " + str(loss))
        step += 1
    print("Optimization Finished!")


In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < 100:
        batch_x, batch_y = [np.expand_dims(pet_data[step%5], axis=4)], label_data[step%5]  # mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if True:
            # Calculate batch loss and accuracy
            loss, y_p, y_a = sess.run([cost, y_pred, y_actual], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            try:    
                prec = precision_score(y_a>0, y_p>.5)
                rec = recall_score(y_a>0, y_p>.5)
                #print(y_p[np.where(y_p>.5)])
                #print(np.shape(np.where(y_p>5))[0])
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Precision= " + \
                      "{:.5f}".format(prec) + ", Training Recall= " + \
                      "{:.5f}".format(rec))
            except Exception:
                import traceback
                traceback.print_exc()
                #print(str(step*batch_size) + " " + str(loss))
        step += 1
    print("Optimization Finished!")
